In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# file_name = '/content/drive/MyDrive/PED/TRabalho_3_2025/MAWI/200701011400.parquet'
file_name = '200701011400.parquet'

# Ler e preparar os dados
df = pd.read_parquet(file_name, engine="pyarrow")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_tcp = df[df['type'] == 'TCP']
df_tcp['second'] = df_tcp['timestamp'].dt.floor('s')
traffic = df_tcp.groupby('second')['size'].sum().reset_index()
traffic.rename(columns={'size': 'bytes'}, inplace=True)

# Escalonamento
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(traffic[['bytes']].values)

# Gerar sequências 'janela'
def create_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

seq_len = 30  # Hiperparâmetro de sequência 'janela'
X, y = create_sequences(data_scaled, seq_len)

# Dividir em treino e teste
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

print(f"Total de sequências: {len(X)}")
print(f"Treinamento: {len(X_train)}")
print(f"Teste:       {len(X_test)}")
print(f"Verificação: soma total = {len(X_train) + len(X_test)}")


2025-06-15 21:42:58.617722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Total de sequências: 871
Treinamento: 696
Teste:       175
Verificação: soma total = 871


/tmp/ipykernel_95228/850901476.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tcp['second'] = df_tcp['timestamp'].dt.floor('s')
